# Resampling Tool

This notebook shows how to resample PROBA-V products (i.e. NDVI, FAPAR...) from 333m resolution to 1km using R-based packages and functions.

# Reading in and preparing data 

When running this notebook on VITO’s servers, the user can follow the directions shown in 
https://nbviewer.jupyter.org/github/VITObelgium/notebook-samples/raw/master/datasets/probav/Reading%20PROBA-V%20using%20R.ipynb to find locations of PROBA-V products and prepare them to be used. Doing so, the data set (netCDF files) will be located and available in the VITO's servers.

Alternatively, when working locally, the R-user can choose to automatically download Copernicus land data (https://land.copernicus.eu/global/) using the functions found in https://github.com/cgls/Copernicus-Global-Land-Service-Data-Download-with-R. 

Once the data set is available, *raster* package functionality is used to prepare it for resampling. 

In [ ]:
library(raster)
#library(knitr)
if(require(knitr) == FALSE){install.packages("knitr", repos = "https://cloud.r-project.org"); library(knitr)} else {library(knitr)}

# ndvi_files is a list of the available files (netCFD or Raster* objects)
ndvi_files <- list(paste0(getwd(), "/ndvi300m_Cat_kk.tif"))
r <- raster(ndvi_files[[1]])

The original netCDF file might contain flagged data corresponding to NoData (NA), etc, which needs to be dealt with. In the following table it can be seen the range of valid values, both DN and physical, for each product and data layer.

In [ ]:
cutoff_method_df <- read.csv(paste0(getwd(), "/Table_cutoff_and_resampleMethod.csv"),
                             stringsAsFactors = FALSE,
                             header = TRUE)
cutoff_method_df <- as.data.frame(gsub("(\\[|\\])", "", as.matrix(cutoff_method_df)))
cutoff_method_df <- as.data.frame(gsub("(\\.\\.\\.)", "_", as.matrix(cutoff_method_df)))

cutoff_method_df$Valid.range.DN.min <- unlist(lapply(cutoff_method_df$Valid.range.DN, function(x) unlist(strsplit(as.character(x), "_"))[[1]]))
cutoff_method_df$Valid.range.DN.max <- unlist(lapply(cutoff_method_df$Valid.range.DN, function(x) unlist(strsplit(as.character(x), "_"))[[2]]))

cutoff_method_df$Valid.range.physical.min <- unlist(lapply(cutoff_method_df$Valid.range.physical, function(x) unlist(strsplit(as.character(x), "_"))[[1]]))
cutoff_method_df$Valid.range.physical.max <- unlist(lapply(cutoff_method_df$Valid.range.physical, function(x) unlist(strsplit(as.character(x), "_"))[[2]]))

cutoff_method_df <- cutoff_method_df[, c("Product","Data.layer.in.file",
                                         "Valid.range.DN.min", "Valid.range.DN.max",
                                         "Valid.range.physical.min", "Valid.range.physical.max",
                                         "Resample.method", "Additional.cutoff")]

kable(cutoff_method_df[, 1:6], caption = "")

cutoff_method_df_ndvi_max <- cutoff_method_df[cutoff_method_df$Data.layer.in.file == "NDVI", c(4, 6)]

For instance, in case of the NDVI products and netCDF files, assigned values > 250 are flagged and need to be converted into NA. When netCDF files are read in as a *Raster*\* object and values transformed into real NDVI values, the cutoff for flagged cells is 0.92. Therefore, all cells > 0.92 have to be set to NA.



In [ ]:
cutoff_flag <- 0.92

r[r > cutoff_flag] <- NA

# Resampling using the aggregation approach

There are several approaches to resample data to a coarser resolution. The area-based aggregation methods are based in grouping rectangular areas of cells of the finer resolution image to create a new map with larger cells. To do that, the function *aggregate()* of the package *raster* needs to know the factor of aggregation. In this case, the factor is 3 as it needs to go from 333m to 1km. In addition, *aggregate()* can perform the calculation using different functions. While the default is the average (*mean()*) it can work also with *modal()*, *max()*, *min()* or *ad hoc* functions (e.g. *closest_to_mean()*, see below). The following table recommends the best suited method for each product/layer.

In [ ]:
kable(cutoff_method_df[, c(1:2, 7)], caption = "")

As for the pixel coordinate definition for both the 300m and the 1km products, no proper aggregation of the finer resolution product can be performed at the minimum and maximum latitude and longitude (see image below). For this reason, the 300m *RasterLayer* object needs to be adjusted accordingly.

![](Pixel_centre_example_CGLS.png)



In [ ]:
if(extent(r)[1] < -180 & extent(r)[2] > 179.997 &
   extent(r)[3] < -59.99554 & extent(r)[4] > 80){  # checking for full product (image)
  extnt_r <- extent(r)
  extnt_r[1] <- extent(r)[1] + (2 * (1 / 336)) # x-min
  extnt_r[2] <- extent(r)[2] - (1 * (1 / 336)) # x-max
  extnt_r[3] <- extent(r)[3] + (1 * (1 / 336))  # y-min
  extnt_r[4] <- extent(r)[4] - (2 * (1 / 336))  # y-max
  r <- crop(r, extnt_r)
}else{
  print("The image is not the full product; therefore, extent needs to be checked")
}

If the user wants to use a subset of the original 300m product, its new extent should match with the 1km product grid. Otherwise, the comparison or any other kind of calculations involving any of the 1km PROVA-B products cannot be made properly. 

So, at this point, the user has two alternative options. On the one hand, the user can provide a 1km *Raster*\* object based on a subset of any PROVA-B product at 1km resolution. 

In [ ]:
ndvi_files_1km <- list(paste0("/Users/xavi_rp/Documents/D6_LPD/NDVI_resample", "/ndvi1km_Cat.tif"))
r_1km <- raster(ndvi_files_1km[[1]])

Alternatively, if no 1km object is provided, a vector with coordinates can be used for subsetting the 300m product. 

In [ ]:
coords4subset <- c(0, 4, 40, 43) # a vector with Long/Lat decimal degrees in the form c(Xmin, Xmax, Ymin, Ymax)

Notice that with both options the extent provided might experience slight changes given that the new extent of the 300m product has to match exactly with the grid coordinates of the 1km products.

Then, the 300m product to be resampled can be geographically subset accordingly to the chosen option and the final extent adjusted if necessary.


In [ ]:
#The following vectors contain Long and Lat coordinates, respectively, of the 1km grid (edges of the cells):
x_ext <- seq((-180 - ((1 / 112) / 2)), 180, (1/112))
y_ext <- seq((80 + ((1 / 112) / 2)), - 60, - (1/112))

In [ ]:
if(exists("r_1km") & all(round(res(r_1km), 7) == round(0.00892857, 7)) &
   round(extent(r_1km)[1], 7) %in% round(x_ext, 7) & 
   round(extent(r_1km)[2], 7) %in% round(x_ext, 7) &
   round(extent(r_1km)[3], 7) %in% round(y_ext, 7) &
   round(extent(r_1km)[4], 7) %in% round(y_ext, 7)){ # r_1km matches with PROBA-V 1km products
  
  r <- crop(r, r_1km)
  
}else if(exists("r_1km") & all(round(res(r_1km), 7) == round(0.00892857, 7)) &
   !all(round(extent(r_1km)[1], 7) %in% round(x_ext, 7) & 
   round(extent(r_1km)[2], 7) %in% round(x_ext, 7) &
   round(extent(r_1km)[3], 7) %in% round(y_ext, 7) &
   round(extent(r_1km)[4], 7) %in% round(y_ext, 7))){  # r_1km exists but does not match with PROBA-V 1km products
  
  stop("Please provide a 1km object adjusted to 1km PROBA-V products")

}else if (!exists("r_1km") & exists("coords4subset") &
          length(coords4subset) == 4 &
          all(coords4subset >= -180) &
          all(coords4subset <= 180)){  # r_1km not provided, but coordinates to subset from
  
  coords4subset <- extent(coords4subset)
  r <- crop(r, coords4subset)
  
}else{
  stop("Please provide either a 1km PROBA-V product or a set of coordinates to cut from")
}


## Adjusting extent
extnt <- extent(r)
  
repeat{# adjusting columns (Lon) at left
  if (round(extnt[1], 7) %in% round(x_ext, 7)) break
  cat("\r", "Please note that 300m object X-min does not overlap 1km object X-min. Keep it in mind for further analysis")
  extnt[1] <- extnt[1] + (1/336)
}

repeat{ # adjusting columns (Lon) at right
  if(round(extnt[2], 7) %in% round(x_ext, 7)) break
  cat("\r", "Please note that 300m object X-max does not overlap 1km object X-max. Keep it in mind for further analysis")
  extnt[2] <- extnt[2] - (1/336)
}

repeat{# adjusting rows (Lat) at top
  if (round(extnt[4], 7) %in% round(y_ext, 7)) break
  cat("\r", "Please note that 300m object Y-max does not overlap 1km object Y-max. Keep it in mind for further analysis")
  extnt[4] <- extnt[4] - (1/336)
}

repeat{ # adjusting rows (Lat) at bottom
  if(round(extnt[3], 7) %in% round(y_ext, 7)) break
  cat("\r", "Please note that 300m object Y-min does not overlap 1km object Y-min. Keep it in mind for further analysis")
  extnt[3] <- extnt[3] + (1/336)
}

r <- crop(r, extnt)

When these checks and coorections are made, the process of resampling itself can go ahead using *aggregate()*. The resample method can be assigned at this point.

In [ ]:
#aggr_method <- "median"
#aggr_method <- "modal"
#aggr_method <- "closest_to_mean"

closest_to_mean <- function(x, ...){
  dts <- list(...)
  if(is.null(dts$na_rm)) dts$na_rm <- TRUE
  x_mean <- mean(x, na.rm = dts$na_rm)
  closest2avrge <- x[order(abs(x - x_mean))][1]
  return(closest2avrge)

}

aggr_method <- "mean"
r300m_resampled1km_Aggr <- aggregate(r,
                                     fact = 3, # from 333m to 1km  
                                     fun = aggr_method, 
                                     na.rm = TRUE, 
                                     filename = '')

Finally, a couple of plots in case the user wants to take a look at the resampled map.

In [ ]:
plot(r, main = 'Original map at 300m')

In [ ]:
plot(r300m_resampled1km_Aggr, main = 'Resampled map to 1km')